In [23]:
import h5py 
import numpy as np
import _mgard as mgard
import time
import cupy

In [24]:
import numpy as np

def psnr(original_data, decompressed_data):
    """
    Calculate Peak Signal-to-Noise Ratio (PSNR) between two 3D arrays with dynamic maximum value determination.

    Args:
        original_data (numpy.ndarray): The original data as a 3D numpy array (float).
        decompressed_data (numpy.ndarray): The decompressed data as a 3D numpy array (float).

    Returns:
        float: The PSNR value.
    """
    # Ensure the input arrays have the same shape
    if original_data.shape != decompressed_data.shape:
        raise ValueError("Input arrays must have the same shape")

    # Calculate the mean squared error (MSE) between the two arrays
    mse = np.mean((original_data - decompressed_data) ** 2)

    # Calculate the dynamic maximum possible value based on the maximum value in the input arrays
    max_possible_value = np.max([np.max(original_data), np.max(decompressed_data)])

    # Calculate the PSNR using the formula: PSNR = 20 * log10(max_possible_value / sqrt(MSE))
    psnr_value = 20 * np.log10(max_possible_value / np.sqrt(mse))

    return psnr_value

def maxError(original_data,decompressed_data):
    if original_data.shape != decompressed_data.shape:
        raise ValueError("Input arrays must have the same shape")
    max_error = abs(np.max(original_data - decompressed_data))
    return max_error

def rmse(original_data,decompressed_data):
    return np.sqrt(np.mean((original_data - decompressed_data) ** 2))


In [25]:

class Slicer:

    # default 引数
    def __init__(self,filename="/scratch/aoyagir/step1_500_test.h5") -> None:
        self.filename = filename
        self.file = h5py.File(filename, 'r')
        self.dataset = self.file['data']
        print(self.dataset.shape)

    # Access specific elements in the concatenated array
    def access_array_element(self,timestep, x, y, z):
        element = self.dataset[timestep, x, y, z]
        return element

    # Access a subset of the concatenated array
    def slice_multiple_step(self, file, timestep_start, timestep_end, x_start, x_end, y_start, y_end, z_start, z_end):
        subset = self.dataset[timestep_start:timestep_end, x_start:x_end, y_start:y_end, z_start:z_end]
        return subset
    
    # slice siingle step
    def slice_single_step(self, timestep,  x_start, x_end, y_start, y_end, z_start, z_end):
        subset = self.dataset[timestep,  x_start:x_end, y_start:y_end, z_start:z_end]
        retsubset = np.squeeze(subset)
        return retsubset

    # slice sigle step by size
    def get_xyz_offset_by_size(self, size):
        # 100MB -> 「100/(sizeof(float))」個のデータ
        sizeFloat = 4 # byte
        return int((size/sizeFloat)**(1/3))



In [26]:
slicer = Slicer("/scratch/aoyagir/step1_256_test_0902.h5")

(257, 1024, 1024, 1024)


In [16]:
# minとmaxの差が大きいタイムステップを探す
for timestep in range(1,256):
    original = slicer.slice_single_step(timestep, 0, 1023, 0, 1023, 0, 1023)
    range = abs(np.max(original) - np.min(original))
    print(f"timestep={timestep}:range={range}")

timestep=1:range=6.909606456756592
timestep=2:range=6.848243236541748
timestep=3:range=6.784726142883301
timestep=4:range=6.737546443939209
timestep=5:range=6.698577404022217
timestep=6:range=6.688233852386475
timestep=7:range=6.671030521392822
timestep=8:range=6.668423175811768
timestep=9:range=6.663591384887695
timestep=10:range=6.657581329345703
timestep=11:range=6.646807670593262
timestep=12:range=6.633123874664307
timestep=13:range=6.615983009338379
timestep=14:range=6.616232872009277
timestep=15:range=6.604423522949219
timestep=16:range=6.587042808532715
timestep=17:range=6.5709333419799805
timestep=18:range=6.555060386657715
timestep=19:range=6.5447564125061035
timestep=20:range=6.534232139587402
timestep=21:range=6.5303874015808105
timestep=22:range=6.516353607177734
timestep=23:range=6.494901657104492
timestep=24:range=6.479382514953613
timestep=25:range=6.471733093261719
timestep=26:range=6.458970546722412
timestep=27:range=6.446679592132568
timestep=28:range=6.43449401855468

KeyboardInterrupt: 

In [27]:
# create a file to write the results
import csv
from datetime import datetime

# Get the current date and time
current_time = datetime.now()

# Format the current date and time as a string
timestamp = current_time.strftime("%Y%m%d_%H%M%S")

# timestep to conduct the benchmark
# timestep = 128 # レンジ5.69
# レンジがでかいタイムステップ 62 レンジ：7.55
timestep = 62

# Create the file name based on the timestamp
csv_file = f'bench_{timestamp}_timestep_{timestep}_psrn_by_tol.csv'

import csv

header = ['tol', 'oriSize','compressedsize','psnr','max_error','rmse','comp_ratio','method',"time","timespte"]

with open(csv_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(header)



In [28]:
# Define the number of repetitions and initialize a list for results

results = []
tols = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1, 0.5, 1]

configGPU = mgard.Config()
configGPU.dev_type = mgard.DeviceType.CUDA

# Wrap your outer loop with tqdm to create a progress bar

for tol in tols: # 100 iteration
    if tol == 0:
        print("passed")
        continue
    print(tol)
    OriginalSize = 1 * 1024 * 1024 * 1024  # 1MB in bytes

    flag = False

    while OriginalSize <= 8000 * 1024 * 1024:  # 4000MB in bytes. 10 iterations
        # Your code here
        load_exe_times = []  # List to store execution times for each repetition
        comp_exe_times = []
        decomp_exe_times = []
        comp_data_size = None

        # get the offset size
        offset = slicer.get_xyz_offset_by_size(OriginalSize)
        
        original = slicer.slice_single_step(timestep, 0, offset, 0, offset, 0, offset)
        start_tiime = time.time()
        compressed = mgard.compress(original, tol, 0, mgard.ErrorBoundType.REL, configGPU)
        end_time = time.time()
        compression_time = end_time - start_tiime
        compressedSize = compressed.nbytes
        
        start_decomp_time = time.time()
        decompressed = mgard.decompress(compressed, configGPU)
        end_decomp_time = time.time()
        decompression_time = end_decomp_time - start_decomp_time

        ratio = original.nbytes/compressedSize

        p = psnr(original,decompressed)
        maxE = maxError(original,decompressed)
        Rmse = rmse(original,decompressed)

        # ['tol', 'oriSize','compressedsize','psnr','comp_ratio','is_quantimized',"time"]
        row_data = [tol, OriginalSize,compressedSize, p,maxE,Rmse,ratio,"MGARD",compression_time + decompression_time,timestep]

        # Write the data row
        with open(csv_file, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(row_data)



        # quantimization
        start_tiime = time.time()
        quantimized = np.float16(original)
        end_time = time.time()
        time_for_quantimizing = end_time - start_tiime
        
        quantimizedSize = quantimized.nbytes
        ratio = original.nbytes/quantimizedSize
        p = psnr(original,quantimized)
        maxE = maxError(original,quantimized)
        Rmse = rmse(original,quantimized)
        row_data = [0,OriginalSize,quantimizedSize,p,maxE,Rmse,ratio,"numpy",time_for_quantimizing,timestep]
        with open(csv_file, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(row_data)
        flag = True

        
        # cupy quantimization # 上と同じなので、省略します。
        # start_tiime = time.time()
        # quantimized = cupy.float16(original)
        # end_time = time.time()
        # time_for_quantimizing = end_time - start_tiime
        
        # quantimizedSize = quantimized.nbytes
        # ratio = original.nbytes/quantimizedSize
        # p = psnr(original,quantimized)
        # maxE = maxError(original,quantimized)
        # row_data = [0,OriginalSize,quantimizedSize,p,maxE,ratio,"cupy",time_for_quantimizing,timestep]
        # with open(csv_file, "a", newline="") as f:
        #     writer = csv.writer(f)
        #     writer.writerow(row_data)

        
        # Double the size
        OriginalSize *= 2
        
# Convert the results list to a NumPy array for easier manipulation
results_array = np.array(results)

1e-05
5e-05
0.0001
0.0005
0.001
0.005
0.01
0.02
0.03
0.05
0.1
0.5
1


In [ ]:
# tol vs original size
# Define the number of repetitions and initialize a list for results

results = []
tols = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1, 0.5, 1]

configGPU = mgard.Config()
configGPU.dev_type = mgard.DeviceType.CUDA

# Wrap your outer loop with tqdm to create a progress bar

for tol in tols: # 100 iteration
    if tol == 0:
        print("passed")
        continue
    print(tol)

    OriginalSize = 4 * 1024 * 1024 * 1024  # 4GiB in bytes

    # get the offset size
    offset = slicer.get_xyz_offset_by_size(OriginalSize)
    original = slicer.slice_single_step(timestep, 0, offset, 0, offset, 0, offset)
    compressed = mgard.compress(original, tol, 0, mgard.ErrorBoundType.REL, configGPU)
    compressedSize = compressed.nbytes
    decompressed = mgard.decompress(compressed, configGPU)
    p = psnr(original,decompressed)
    
    row_data = [tol, OriginalSize,compressedSize, p]

    # Write the data row
    with open(csv_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(row_data)

    # Double the size
    OriginalSize *= 2
    print(row_data)

# Convert the results list to a NumPy array for easier manipulation
results_array = np.array(results)